In [7]:
from ultralytics import YOLO
from utils import *
from IPython.display import Image, display, Video
from roboflow import Roboflow
import os
os.environ['CUDA_VISIBLE_DEVICES'] = [0,1]

TypeError: str expected, not list

In [6]:
model = YOLO("weights/detect_large_v2.pt")
model.val(device = "cuda")

Ultralytics YOLOv8.0.207 🚀 Python-3.9.7 torch-2.1.0+cu121 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 2
os.environ['CUDA_VISIBLE_DEVICES']: None


In [1]:
import torch


In [9]:
torch.cuda.is_available()

False